In [10]:
# import TensorFlow as TF
import pandas as pd
import math
import matplotlib.pylab as plt
import numpy as np
import statistics

from sklearn import datasets
from sklearn.impute import SimpleImputer # used for handling missing data


In [2]:

df = pd.read_csv('data.csv')
data_top = df.head()
copy = df.copy() 

# 1 - Remove rows where the target column (Scope1) is NaN.

#copy = copy[copy['EmissionDataPoints.CO2EquivalentsEmissionDirectScope1'].notna()] # drop all rows with NaN in target column
#copy = copy.reset_index() 

# 2 - Replace all NaN from the attribute columns and replace for mean of that column

# copy = copy.fillna(copy.mean())
# col_names = copy.columns
# sub2[col_names].fillna((sub2[col_names].mean()), inplace=True)

# create dictionary with 
def create_dict(target_var, df): 
    org_ids = list(df[target_var]) 
    list_set = set(org_ids) # get unique elements
    unique = list(list_set) # place them in a list 
    df = df.groupby([target_var]) 
    
    dictionary = {} 
    for item in unique:
        df_id = df.get_group(item)
        df_id["StatementDetails.FinancialPeriodEndDate"] = pd.to_datetime(df_id["StatementDetails.FinancialPeriodEndDate"])
                
        df_id = df_id.sort_values(by="StatementDetails.FinancialPeriodEndDate")
        dictionary[item] = df_id
    
    return dictionary

dict_id = create_dict('organization_id', copy)
print(dict_id[14346])

#print(copy.organzation_id)

#key_list = list(GTAP_sector_dict.keys())

#print(key_list)

#print(GTAP_sector_dict)

# in the dictionary, we want to replace all the NaN of the values for the means of those columns.


# # check missing values >> columns with lots of missing values are not useful. 
# missing_values_df = copy.isnull().sum().sort_values(ascending=False).head()
# print(missing_values_df)

# # replace NaN in column for mean of column with Imputer 
# imp = SimpleImputer(missing_values='NaN', strategy='mean')
# imp = imp.fit(copy)



#print(replace_NaN(GTAP_sector_dict))    



<ipython-input-2-dba6adb367b0>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_id["StatementDetails.FinancialPeriodEndDate"] = pd.to_datetime(df_id["StatementDetails.FinancialPeriodEndDate"])


       organization_id StatementDetails.FinancialPeriodEndDate  \
66052            14346               2018-12-31 00:00:00+00:00   
29557            14346               2019-12-31 00:00:00+00:00   

       StatementDetails.FinancialPeriodFiscalYear  \
66052                                        2018   
29557                                        2019   

       StatementDetails.FinancialPeriodIsIncomplete  \
66052                                         False   
29557                                         False   

      StatementDetails.EsgOrAtdIndicator  EmissionDataPoints.AccidentalSpills  \
66052                                ESG                                  NaN   
29557                                ESG                                  NaN   

      EmissionDataPoints.AnalyticCO2EstimationMethod  \
66052                                         Median   
29557                                       Reported   

       EmissionDataPoints.AnalyticEstimatesCO2EquivalentsEmiss

In [3]:
# function for filling NaN's
def fill_nan(df, column_name):
    mean_value=df[column_name].mean()
    df[column_name].fillna(value=mean_value, inplace=True)
    return df

# function for determining percentage NaN
def ratio_nan(df, column_name):
    ratio = df[column_name].isnull().sum() / len(df)
    return ratio

# function to loop over org_ids and creat df with ratio's nan in column
def dict_ratios(dictionary, column_name):
    dict_ratios = {}
    for key in dictionary:
        ratio_key = ratio_nan(dictionary[key], column_name)
        dict_ratios[key] = ratio_key
    return dict_ratios

#print(np.array(list(dict_ratios(dict_id, 'EmissionDataPoints.CO2EquivalentsEmissionDirectScope1').values())).mean())
#print(dict_ratios(dict_id, 'EmissionDataPoints.CO2EquivalentsEmissionDirectScope1'))

def df_clean(df, column_name):
    df = df.reset_index(drop=True)
    
    for i in range(1, len(df[column_name])-2):
        if math.isnan(df.at[i, column_name]):
            if not math.isnan(df.at[i-1, column_name]):
                if not math.isnan(df.at[i+1, column_name]):
                    mean = (df.at[i+1, column_name] + df.at[i-1, column_name])/2
                    df.at[i, column_name] = mean
                    
    if math.isnan(df.at[0, column_name]):
        if not math.isnan(df.at[1, column_name]):
            df.at[0, column_name] = df.at[1, column_name]
            
    last_point = len(df[column_name]) - 1
    
    if math.isnan(df.at[last_point, column_name]):
        if not math.isnan(df.at[last_point-1, column_name]):
            df.at[last_point, column_name] = df.at[last_point-1, column_name]
    
    df = df[df[column_name].notna()]
    return df

def dict_clean_better(dictionary, column_name, ratio):
    dict_clean = {}
    for key in dictionary:
        ratio_key = ratio_nan(dictionary[key], column_name)
        if ratio_key <= ratio:
            dict_clean[key] = df_clean(dictionary[key], column_name)
        else:
            df_key = dictionary[key]
            df_key = df_key[df_key[column_name].notna()]
            dict_clean[key] = df_key
    return dict_clean

def dict_clean_mean(dictionary, column_name, ratio):
    dict_clean = {}
    for key in dictionary:
        ratio_key = ratio_nan(dictionary[key], column_name)
        if ratio_key <= ratio:
            dict_clean[key] = fill_nan(dictionary[key], column_name)
        else:
            df_key = dictionary[key]
            df_key = df_key[df_key[column_name].notna()]
            dict_clean[key] = df_key
    return dict_clean


dict_nan = dict_clean_better(dict_id, 'EmissionDataPoints.CO2EquivalentsEmissionDirectScope1', 0.2)


# print(ratio_nan(dict_id[6230], 'EmissionDataPoints.CO2EquivalentsEmissionDirectScope1'))

# df_6230 = dict_id[6230]
# df_clean_6230 = dict_nan[6230]

df_72 = dict_id[72]
df_clean_72 = dict_nan[72]

print(df_72['EmissionDataPoints.CO2EquivalentsEmissionDirectScope1'])
print(df_clean_72['EmissionDataPoints.CO2EquivalentsEmissionDirectScope1'])




57574        NaN
25387    39982.0
20060    42051.0
Name: EmissionDataPoints.CO2EquivalentsEmissionDirectScope1, dtype: float64
25387    39982.0
20060    42051.0
Name: EmissionDataPoints.CO2EquivalentsEmissionDirectScope1, dtype: float64


In [9]:
#copy.groupby("organization_id")['EmissionDataPoints.CO2EquivalentsEmissionDirectScope1'].plot(x='StatementDetails.FinancialPeriodFiscalYear',y='EmissionDataPoints.CO2EquivalentsEmissionDirectScope1')
# .mean()

#print(list(copy.columns))


df = df[df['organization_id'].notna()]


def find_correlation(df, target_value,):
    # way to determine if columns have correlation
    df_target = df[target_value]
    corr = df.corrwith(df_target)
    return corr

print(dict(find_correlation(dict_nan[6230], 'EmissionDataPoints.CO2EquivalentsEmissionDirectScope1')))



{'organization_id': nan, 'StatementDetails.FinancialPeriodFiscalYear': 0.8999832657958088, 'StatementDetails.FinancialPeriodIsIncomplete': nan, 'EmissionDataPoints.AccidentalSpills': nan, 'EmissionDataPoints.AnalyticEstimatesCO2EquivalentsEmissionTotal': 0.8337346839129687, 'EmissionDataPoints.BiodiversityImpactReduction': nan, 'EmissionDataPoints.CarbonOffsetsCredits': nan, 'EmissionDataPoints.CementCO2EquivalentsEmission': nan, 'EmissionDataPoints.ClimateChangeCommercialRisksOpportunities': nan, 'EmissionDataPoints.CO2EquivalentsEmissionDirectScope1': 1.0, 'EmissionDataPoints.CO2EquivalentsEmissionIndirectScope3': 0.9066992757621494, 'EmissionDataPoints.CO2EquivalentsEmissionIndirectScope2': 0.5656518881672119, 'EmissionDataPoints.CO2EquivalentsEmissionTotal': 0.8337346839129687, 'EmissionDataPoints.EmissionReductionTargetPercentage': nan, 'EmissionDataPoints.EmissionReductionTargetYear': nan, 'EmissionDataPoints.EmissionsTrading': -0.7489916010196348, 'EmissionDataPoints.EmsCertifie

In [19]:
from datetime import datetime

from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer # used for handling missing data
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import mean_absolute_error

dict_id = create_dict('GTAP sector', copy)
air = dict_id['Air transport']
# print(air)
Y = air['EmissionDataPoints.CO2EquivalentsEmissionDirectScope1']
X = air.loc[:, ['StatementDetails.FinancialPeriodFiscalYear']]

X = np.array(X)
# X = X.reshape(-1,1) # alleen voor 1 input variable
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.33, random_state=42)
degree = 5
polyreg=make_pipeline(PolynomialFeatures(degree), LinearRegression())
polyreg.fit(X_train,Y_train)
predictions = polyreg.predict(X_test)

# print("Coefficients: \n", polyreg.coef_)
# The mean squared error
print("Mean absolute error: %.2f" % mean_absolute_error(Y_test, predictions))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(Y_test, predictions))

plt.scatter(X_test[:, 0], predictions, label='prediction')
plt.scatter(X_test[:, 0], Y_test, label='test data')

plt.xlabel("year")
plt.ylabel("CO2 emission")
plt.legend()
plt.show()

<ipython-input-2-dba6adb367b0>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_id["StatementDetails.FinancialPeriodEndDate"] = pd.to_datetime(df_id["StatementDetails.FinancialPeriodEndDate"])


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [17]:
def manage_outliers(df, column_name, d_max):
    df = df.reset_index(drop=True)    
    for i in range(len(df[column_name])-1):
        x = df.at[i, column_name]
        x_hat = df[column_name].mean()
        stddv = statistics.stdev(df[column_name])
        d_x = (x - x_hat)/stddv
        if d_x > d_max:
            df.drop(i)
    return df

dict_id = create_dict('GTAP sector', copy)
air = dict_id['Air transport']
air = df_clean(air, 'EmissionDataPoints.CO2EquivalentsEmissionDirectScope1')

#print(air['EmissionDataPoints.CO2EquivalentsEmissionDirectScope1'])

air_outliers = manage_outliers(air, 'EmissionDataPoints.CO2EquivalentsEmissionDirectScope1', 0.5)
#print(air_outliers['EmissionDataPoints.CO2EquivalentsEmissionDirectScope1'])

<ipython-input-2-dba6adb367b0>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_id["StatementDetails.FinancialPeriodEndDate"] = pd.to_datetime(df_id["StatementDetails.FinancialPeriodEndDate"])


11      7000000.0
18      9700000.0
22      6700000.0
28     10200000.0
32      6130968.0
          ...    
548    16315296.0
552     3176634.0
553     9276620.0
556     1995197.0
572    11020000.0
Name: EmissionDataPoints.CO2EquivalentsEmissionDirectScope1, Length: 364, dtype: float64
0       7000000.0
1       9700000.0
2       6700000.0
3      10200000.0
4       6130968.0
          ...    
359    16315296.0
360     3176634.0
361     9276620.0
362     1995197.0
363    11020000.0
Name: EmissionDataPoints.CO2EquivalentsEmissionDirectScope1, Length: 364, dtype: float64


In [18]:
Y = air['EmissionDataPoints.CO2EquivalentsEmissionDirectScope1']
X = air.loc[:, ['StatementDetails.FinancialPeriodFiscalYear']]

X = np.array(X)
# X = X.reshape(-1,1) # alleen voor 1 input variable
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.33, random_state=42)
degree = 5
polyreg=make_pipeline(PolynomialFeatures(degree), LinearRegression())
polyreg.fit(X_train,Y_train)
predictions = polyreg.predict(X_test)

# print("Coefficients: \n", polyreg.coef_)
# The mean squared error
print("Mean absolute error: %.2f" % mean_absolute_error(Y_test, predictions))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(Y_test, predictions))

plt.scatter(X_test[:, 0], predictions, label='prediction')
plt.scatter(X_test[:, 0], Y_test, label='test data')

plt.xlabel("year")
plt.ylabel("CO2 emission")
plt.legend()
plt.show()

NameError: name 'train_test_split' is not defined